## import the data from the database 

This is my attempt to make a connection between the postgres database and this file
It seems to work up until the import config from config portion 
I'm not sure if I don't have my environment set up correctly or what the issue is 
but it is needed because the program will recognize the class we are trying to use as 
a module


In [1]:
#import the password from a seperate file that is ignored on github
# from config import password
# import dependencies 
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

/Users/clarerobbins/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from getpass import getpass 
password = getpass ('Enter database password')

try: 
    conn = psycopg2.connect(
        host="localhost",
        database = "Tennessee_Tornadoes",
        user="postgres",
        password=password)
    print ('pyscopg2 connection:', conn)
    
except Exception as err:
    print ('psycopg2 connect() ERROR:', err)
    connect = None


Enter database password········
pyscopg2 connection: <connection object at 0x7f90e08c3440; dsn: 'user=postgres password=xxx dbname=Tennessee_Tornadoes host=localhost', closed: 0>


In [3]:
cr = conn.cursor()
cr.execute('SELECT * FROM tennessee_tornadoes;')
tmp = cr.fetchall()

In [5]:
#extract the  column names
col_names = []
for db in cr.description:
    col_names.append(db[0])

In [6]:
# Create the dataframe, passing in the list of col_names extracted from the description
df = pd.DataFrame(tmp, columns=col_names)
df.head()

,fid,om,yr,mo,dy,date,time,tz,st,stf,...,fat,loss,closs,slat,slon,elat,elon,len,wid,shape_leng
0,23025,303811,2011,4,9,2011-04-09,14:35:00,3,TN,47,...,0,0.02,0.0,36.2971,-82.3174,36.2934,-82.3021,0.89,50,0.015741
1,23027,291955,2011,4,9,2011-04-09,16:00:00,3,TN,47,...,0,0.075,0.0,36.3,-82.4341,36.3,-82.4341,3.0,150,0
2,23028,303792,2011,4,9,2011-04-09,16:08:00,3,TN,47,...,0,0.01,0.0,36.3037,-82.3923,36.3099,-82.3846,0.61,100,0.009886
3,23029,303819,2011,4,9,2011-04-09,16:15:00,3,TN,47,...,0,0.015,0.0,36.0424,-82.5568,36.0477,-82.5393,1.04,50,0.018285
4,23044,347401,2011,4,27,2011-04-27,19:07:00,3,TN,47,...,0,0.005,0.0,36.2255,-83.057,36.2263,-83.0486,0.5,70,0.008438


In [8]:
# #!/usr/bin/python
# from configparser import ConfigParser


# def config(filename='database.ini', section='postgresql'):
#     # create a parser
#     parser = ConfigParser()
#     # read config file
#     parser.read(filename)

#     # get section, default to postgresql
#     db = {}
#     if parser.has_section(section):
#         params = parser.items(section)
#         for param in params:
#             db[param[0]] = param[1]
#     else:
#         raise Exception('Section {0} not found in the {1} file'.format(section, filename))

#     return db


In [6]:
# from config import config

ImportError: cannot import name 'config' from 'config' (/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/config/__init__.py)

In [ ]:
# #!/usr/bin/python

# def connect():
#     """ Connect to the PostgreSQL database server """
#     conn = None
#     try:
#         # read connection parameters
#         params = config()

#         # connect to the PostgreSQL server
#         print('Connecting to the PostgreSQL database...')
#         conn = psycopg2.connect(**params)

#         # create a cursor
#         cur = conn.cursor()
#     # execute a statement
#         print('PostgreSQL database version:')
#         cur.execute('SELECT version()')

#         # display the PostgreSQL database server version
#         db_version = cur.fetchone()
#         print(db_version)
       
#     # close the communication with the PostgreSQL
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#     finally:
#         if conn is not None:
#             conn.close()
#             print('Database connection closed.')


# if __name__ == '__main__':
#     connect()
    
    


In [ ]:
#python connect.py

### Prepare Data for the ML

In [ ]:
# Create our features

target = ''
X = pd.get_dummies(df.drop(columns=loss))


# Create our target
y = df[loss]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

### Run the first model

target = loss

In [ ]:


model = TBD 


model = model.fit(X_train, y_train)
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
# Display the confusion matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")
# Print the imbalanced classification report
print(f"Classification Report:\n{classification_report_imbalanced(y_test, y_pred)}")

### Run the second model 

target = location 

In [ ]:


model = TBD 


model = model.fit(X_train, y_train)
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
# Display the confusion matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")
# Print the imbalanced classification report
print(f"Classification Report:\n{classification_report_imbalanced(y_test, y_pred)}")

### Run the third model 

target = Magnitude 

In [ ]:
model = TBD 


model = model.fit(X_train, y_train)
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
print(f"The accuracy score: \n{balanced_accuracy_score(y_test, y_pred)}\n")
# Display the confusion matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")
# Print the imbalanced classification report
print(f"Classification Report:\n{classification_report_imbalanced(y_test, y_pred)}")